# Ocean Heat Content

$OHC = \Delta T \times c_p \times \rho \times V$

unit conversion to SI
- $c_p$: erg/g/K = 1e-7J / 1e-3kg / K = 1e-4 J/kg/K $\rightarrow$ 3996 J/kg/K
- $\rho$: g/cm^3 = 1e3 kg/m^3

In [ ]:
import sys
sys.path.append("..")
import scipy as sp
import numpy as np
import xarray as xr
import seaborn as sns
import cmocean
import cartopy
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

In [ ]:
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2
%aimport - numpy - scipy - matplotlib.pyplot

In [ ]:
from OHC import OHC_integrals, trend_global_levels
from maps import map_robinson
from grid import create_dz_mean, create_tdepth
from paths import CESM_filename, path_samoc, file_ex_ocn_ctrl, file_ex_ocn_rcp, path_results
from regions import boolean_mask, regions_dict
from constants import cp_sw, spy
from OHC_plots import plot_global_integrals, plot_global_integrals_detr, plot_global_integrals_diff, plot_levels_trend
from OHC_plots import Hovmoeller_global_depth, Hovmoeller_basins_depth
from timeseries import IterateOutputCESM
from xr_integrate import  xr_vol_int
from xr_DataArrays import create_xr_DataArray, xr_DZ, xr_AREA, xr_HTN, xr_LATS
from xr_regression import xr_linear_trend, xr_linear_trends_2D, zonal_trend, zonal_levels_trend, ocn_field_regression

In [ ]:
ds = xr.open_dataset(file_ex_ocn_ctrl, decode_times=False)
MASK = ds.REGION_MASK

In [ ]:
MASK.plot()

In [ ]:
km = 42

In [ ]:
ctrl   = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_ctrl_Global_Ocean.nc'  , decode_times=False)
ctrl_A = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_ctrl_Atlantic_Ocean.nc', decode_times=False)
ctrl_P = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_ctrl_Pacific_Ocean.nc' , decode_times=False)
ctrl_I = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_ctrl_Indian_Ocean.nc'  , decode_times=False)
ctrl_M = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_ctrl_Mediterranean.nc' , decode_times=False)
ctrl_S = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_ctrl_Southern_Ocean.nc', decode_times=False)
ctrl_N = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_ctrl_Arctic_Ocean.nc', decode_times=False)

rcp    = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_rcp_Global_Ocean.nc'   , decode_times=False)
rcp_A  = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_rcp_Atlantic_Ocean.nc' , decode_times=False)
rcp_P  = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_rcp_Pacific_Ocean.nc'  , decode_times=False)
rcp_I  = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_rcp_Indian_Ocean.nc'   , decode_times=False)
rcp_M  = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_rcp_Mediterranean.nc'  , decode_times=False)
rcp_S  = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_rcp_Southern_Ocean.nc' , decode_times=False)
rcp_N  = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_rcp_Arctic_Ocean.nc' , decode_times=False)


In [ ]:
# something is still wrong in CTRL year 205
for ds in [ctrl, ctrl_A, ctrl_P, ctrl_I, ctrl_S, ctrl_M, ctrl_N]:
    for field in ['OHC_global', 'OHC_global_levels', 'OHC_zonal', 'OHC_zonal_levels']:
        ds[field][105] = (ds[field].sel({'time':204*365}) +
                                           ds[field].sel({'time':206*365}) )/2

In [ ]:
ctrl_t = np.array(ctrl.time.values/365, dtype=int)
rcp_t  = np.array(rcp.time.values/365 , dtype=int)
n_ctrl, n_rcp = len(ctrl_t), len(rcp_t)
print(n_ctrl, n_rcp)

In [ ]:
dz_mean = create_dz_mean(domain='ocn')
tdepth = create_tdepth('ocn')

## global integral

In [ ]:
OHC_global_init = ctrl.OHC_global[100].item()

In [ ]:
ctrl_times = ctrl.time/365
rcp_times  = rcp.time/365-1800
lfit_ctrl  = np.polyfit(ctrl_times, ctrl.OHC_global, 1)
lfit_rcp   = np.polyfit(rcp_times , rcp.OHC_global , 1)
qfit_ctrl  = np.polyfit(ctrl_times, ctrl.OHC_global, 2)
qfit_rcp   = np.polyfit(rcp_times , rcp.OHC_global , 2)
OHC_global_lfit_ctrl = lfit_ctrl[1] + lfit_ctrl[0]*ctrl_times
OHC_global_qfit_ctrl = qfit_ctrl[2] + qfit_ctrl[1]*ctrl_times + qfit_ctrl[0]*ctrl_times**2
OHC_global_lfit_rcp  = lfit_rcp [1] + lfit_rcp [0]*rcp_times
OHC_global_qfit_rcp  = qfit_rcp [2] + qfit_rcp [1]*rcp_times  + qfit_rcp[0]*rcp_times**2
OHC_global_ldetr_ctrl = ctrl.OHC_global - OHC_global_lfit_ctrl
OHC_global_qdetr_ctrl = ctrl.OHC_global - OHC_global_qfit_ctrl
OHC_global_ldetr_rcp  = rcp.OHC_global  - OHC_global_lfit_rcp
OHC_global_qdetr_rcp  = rcp.OHC_global  - OHC_global_qfit_rcp

In [ ]:
f = plt.figure(figsize=(8,5))
ax = f.add_axes([0.13,0.13,.85,.85])
plt.tick_params(labelsize=14)
plt.axhline(0, c='k', lw=.5)
L1, = plt.plot(ctrl_times, (OHC_global_qfit_ctrl-OHC_global_init)/1e21, c='k', label='quad. fits')
L2, = plt.plot(rcp_times , (OHC_global_qfit_rcp -OHC_global_init)/1e21, c='k', label='quad. fit')
L3, = plt.plot(ctrl_times, (ctrl.OHC_global.values-OHC_global_init)/1e21, lw=2, c=f'C{0}', label='CTRL')
L4, = plt.plot(rcp_times , (rcp.OHC_global.values -OHC_global_init)/1e21, lw=2, c=f'C{1}', label='RCP' )
L5, = plt.plot(ctrl_times, OHC_global_qdetr_ctrl/1e21, ls='--', lw=2, c=f'C{0}', label='CTRL quad. detrended')
L6, = plt.plot(rcp_times , OHC_global_qdetr_rcp /1e21, ls='--', lw=2, c=f'C{1}', label='RCP  quad. detrended')
plt.xlabel('time [years]', fontsize=16)
plt.ylabel('Ocean Heat Content [ZJ]', fontsize=16)
plt.legend(handles=[L1, L3, L4, L5, L6], fontsize=16)
plt.tight_layout()
plt.savefig(f'{path_results}/OHC/OHC_global_timeseries')

In [ ]:
f = plt.figure(figsize=(8,5))
ax = f.add_axes([0.13,0.13,.85,.85])
plt.tick_params(labelsize=14)
plt.axhline(0, c='k', lw=.5)
plt.plot(ctrl_times,OHC_global_qdetr_ctrl/1e21, lw=3, label='CTRL quad. detrended')
plt.plot(rcp_times ,OHC_global_qdetr_rcp /1e21, lw=3, label='RCP quad. detrended')
plt.plot(ctrl_times, (ctrl.OHC_global-ctrl.OHC_global.shift(time=1))/1e21, ls=':', lw=2, c=f'C{0}', label='$\Delta$OHC CTRL')
plt.plot(rcp_times , (rcp.OHC_global -rcp.OHC_global.shift( time=1))/1e21, ls=':', lw=2, c=f'C{1}', label='$\Delta$OHC RCP')
plt.ylim((-25,25))
plt.xlabel('time [years]', fontsize=14)
plt.ylabel('detrended OHC, OHC change [ZJ]', fontsize=16)
plt.legend(loc=4, fontsize=16, frameon=False)
plt.tight_layout()
plt.savefig(f'{path_results}/OHC/OHC_global_timeseries_detrended')

In [ ]:
# detrended rcp timeseries to find location of hiatus
dss = [ctrl, ctrl_A, ctrl_P, ctrl_I, ctrl_S, ctrl_M]
plot_global_integrals(dss, 'ctrl')
plot_global_integrals_diff(dss, 'ctrl')
plot_global_integrals_detr(dss, 'ctrl')

dss = [rcp, rcp_A, rcp_P, rcp_I, rcp_S, rcp_M]
plot_global_integrals(dss, 'rcp')
plot_global_integrals_diff(dss, 'rcp')
plot_global_integrals_detr(dss, 'rcp')

## Levels

In [ ]:
ctrl_levels_trend_G = trend_global_levels(ctrl)
ctrl_levels_trend_A = trend_global_levels(ctrl_A)
ctrl_levels_trend_P = trend_global_levels(ctrl_P)
ctrl_levels_trend_I = trend_global_levels(ctrl_I)
ctrl_levels_trend_S = trend_global_levels(ctrl_S)
ctrl_levels_trend_M = trend_global_levels(ctrl_M)

rcp_levels_trend_G = trend_global_levels(rcp)
rcp_levels_trend_A = trend_global_levels(rcp_A)
rcp_levels_trend_P = trend_global_levels(rcp_P)
rcp_levels_trend_I = trend_global_levels(rcp_I)
rcp_levels_trend_S = trend_global_levels(rcp_S)
rcp_levels_trend_M = trend_global_levels(rcp_M)

In [ ]:
f = plt.figure(figsize=(8,5))
ax = f.add_axes([0.10,0.13,.88,.85])
plt.tick_params(labelsize=14)
plt.axvline(0, c='k', lw=.5)
plt.plot(ctrl_levels_trend_G, -tdepth/1e3, lw=3, label='CTRL')
plt.plot(rcp_levels_trend_G , -tdepth/1e3, lw=3, label='RCP')
plt.xlabel('OHC trend [J/m/yr]', fontsize=16)
plt.ylabel('depth [km]', fontsize=16)
plt.legend(fontsize=16)
plt.tight_layout()
plt.savefig(f'{path_results}/OHC/OHC_trend_depth')

In [ ]:
das = [ctrl_levels_trend_G,
       ctrl_levels_trend_A,
       ctrl_levels_trend_P,
       ctrl_levels_trend_I,
       ctrl_levels_trend_S,
       ctrl_levels_trend_M,
      ]
plot_levels_trend(das=das, run='ctrl')

das = [rcp_levels_trend_G,
       rcp_levels_trend_A,
       rcp_levels_trend_P,
       rcp_levels_trend_I,
       rcp_levels_trend_S,
       rcp_levels_trend_M,
      ]
plot_levels_trend(das=das, run='rcp')

das = [rcp_levels_trend_G-ctrl_levels_trend_G,
       rcp_levels_trend_A-ctrl_levels_trend_A,
       rcp_levels_trend_P-ctrl_levels_trend_P,
       rcp_levels_trend_I-ctrl_levels_trend_I,
       rcp_levels_trend_S-ctrl_levels_trend_S,
       rcp_levels_trend_M-ctrl_levels_trend_M,
      ]
plot_levels_trend(das=das, run='rcp-ctrl')

In [ ]:
ctrl_global_level_lin_fit = np.zeros((2, km))
rcp_global_level_lin_fit  = np.zeros((2, km))
rcp_global_level_quad_fit = np.zeros((3, km))
for k in range(km):
    ctrl_global_level_lin_fit[:,k] = np.polyfit(ctrl_times, ctrl.OHC_global_levels[:,k]/dz_mean[k], 1)
    rcp_global_level_lin_fit[:,k]  = np.polyfit(rcp_times , rcp.OHC_global_levels[:,k] /dz_mean[k], 1)
    rcp_global_level_quad_fit[:,k] = np.polyfit(rcp_times , rcp.OHC_global_levels[:,k] /dz_mean[k], 2)

# plot fitting parameters per depth    
f,ax = plt.subplots(1,2)
ax[0].plot(ctrl_global_level_lin_fit[0,:])
ax[0].plot(rcp_global_level_lin_fit[0,:])
ax[0].set_xlabel('slope')
ax[1].plot(ctrl_global_level_lin_fit[1,:])
ax[1].plot(rcp_global_level_lin_fit[1,:])
ax[1].set_xlabel('offset')

In [ ]:
ctrl_global_levels_lin = np.zeros((n_ctrl, km))
rcp_global_levels_lin  = np.zeros((n_rcp, km))
rcp_global_levels_quad = np.zeros((n_rcp, km))
for t in range(n_ctrl):
    ctrl_global_levels_lin[t,:] = ctrl_global_level_lin_fit[0,:]*t + ctrl_global_level_lin_fit[1,:]
    if t<n_rcp:
        rcp_global_levels_lin[t,:] = rcp_global_level_lin_fit[0,:]*t + rcp_global_level_lin_fit[1,:]
        rcp_global_levels_quad[t,:] = rcp_global_level_quad_fit[0,:]*t**2 + rcp_global_level_quad_fit[1,:]*t + rcp_global_level_quad_fit[2,:]    

In [ ]:
ctrl.OHC_global_levels

In [ ]:
Hovmoeller_global_depth(ctrl, detrend='lin' , fn=f'{path_results}/OHC/OHC_Hovmoeller_depth_global_ctrl')
Hovmoeller_global_depth(rcp , detrend='lin' , fn=None)
Hovmoeller_global_depth(rcp , detrend='quad', fn=f'{path_results}/OHC/OHC_Hovmoeller_depth_global_rcp')

In [ ]:
dss_ctrl = [ctrl_S, ctrl_A, ctrl_P, ctrl_I]
dss_rcp  = [rcp_S, rcp_A, rcp_P, rcp_I]
Hovmoeller_basins_depth(dss_ctrl, detrend='lin' , fn=f'{path_results}/OHC/OHC_Hovmoeller_depth_basins_ctrl')
Hovmoeller_basins_depth(dss_rcp , detrend='lin' , fn=None) 
Hovmoeller_basins_depth(dss_rcp , detrend='quad', fn=f'{path_results}/OHC/OHC_Hovmoeller_depth_basins_rcp' )

## spatial trends of vertically integrated OHC

In [ ]:
%%time
    OHC_vert_trend_ctrl = ocn_field_regression(ctrl.OHC_vertical)/spy  # [J/m^2/year] to [W/m^2]
    OHC_vert_trend_rcp  = ocn_field_regression(rcp.OHC_vertical )/spy

In [ ]:
label  = r'OHC trend [W/m$^2$]'
domain = 'ocn_T'
maxv   = 6
cmap   = cmocean.cm.balance

fn = f'{path_results}/OHC/OHC_trend_vert_int_map_ctrl'
xa = OHC_vert_trend_ctrl
f, ax = map_robinson(xa=xa, domain=domain, cmap=cmap, minv=-maxv, maxv=maxv, label=label, filename=fn)

fn = f'{path_results}/OHC/OHC_trend_vert_int_map_rcp'
xa = OHC_vert_trend_rcp
f, ax = map_robinson(xa=xa, domain=domain, cmap=cmap, minv=-maxv, maxv=maxv, label=label, filename=fn)

## zonal integrals

In [ ]:
lats = ctrl.TLAT_bins

f = plt.figure(figsize=(8,5))
ax = f.add_axes([0.13,0.13,.84,.82])
plt.tick_params(labelsize=14)
plt.axhline(0, c='k', lw=.5)

y1 = ctrl_S.OHC_zonal.mean(dim='time').values
y2 = 0
S  = plt.fill_between(lats, y1, y2, color='C3', alpha=.5, label='Southern')

y1 = (np.nan_to_num(ctrl_S.OHC_zonal)+ctrl_A.OHC_zonal).mean(dim='time').values
y2 = np.nan_to_num(ctrl_S.OHC_zonal.mean(dim='time').values)
A  = plt.fill_between(lats, y1, y2, color='C0', alpha=.5, label='Atlantic')

y1 = (np.nan_to_num(ctrl_S.OHC_zonal)+ctrl_A.OHC_zonal+ctrl_P.OHC_zonal).mean(dim='time').values
y2 = (np.nan_to_num(ctrl_S.OHC_zonal)+ctrl_A.OHC_zonal                 ).mean(dim='time').values
P  = plt.fill_between(lats, y1, y2, color='C1', alpha=.5, label='Pacific')

y1 = (np.nan_to_num(ctrl_S.OHC_zonal)+ctrl_A.OHC_zonal+ctrl_P.OHC_zonal+ctrl_I.OHC_zonal).mean(dim='time').values
y2 = (np.nan_to_num(ctrl_S.OHC_zonal)+ctrl_A.OHC_zonal+ctrl_P.OHC_zonal                 ).mean(dim='time').values
I  = plt.fill_between(lats, y1, y2, color='C2', alpha=.5, label='Indian')

y1 = (ctrl_A.OHC_zonal+ctrl_P.OHC_zonal+ctrl_M.OHC_zonal).mean(dim='time').values
y2 = (ctrl_A.OHC_zonal+ctrl_P.OHC_zonal                 ).mean(dim='time').values
M  = plt.fill_between(lats, y1, y2, color='C4', alpha=.5, label='Mediterranean')

G, = plt.plot(lats , ctrl.OHC_zonal.mean(dim='time').values, c='k', lw=3, label='global integral')

plt.text(.98,.9, f'CTRL mean', ha='right', transform=ax.transAxes, fontsize=16)



plt.xticks(np.arange(-90,91,30))
plt.xlim((-92,92))
plt.ylim((0,2.6e18))
plt.xlabel('latitude', fontsize=16)
plt.ylabel('zonal OHC integral [J/m]', fontsize=16)
plt.legend(handles=[G, M, S, A, P, I], fontsize=16, loc=2, frameon=False)
plt.tight_layout()
plt.savefig(f'{path_results}/OHC/OHC_zonal_mean')

In [ ]:
ctrl_zonal_trend_G = zonal_trend(ctrl)
ctrl_zonal_trend_A = zonal_trend(ctrl_A)
ctrl_zonal_trend_P = zonal_trend(ctrl_P)
ctrl_zonal_trend_I = zonal_trend(ctrl_I)
ctrl_zonal_trend_M = zonal_trend(ctrl_M)
ctrl_zonal_trend_S = zonal_trend(ctrl_S)

rcp_zonal_trend_G  = zonal_trend(rcp)
rcp_zonal_trend_A  = zonal_trend(rcp_A)
rcp_zonal_trend_P  = zonal_trend(rcp_P)
rcp_zonal_trend_I  = zonal_trend(rcp_I)
rcp_zonal_trend_M  = zonal_trend(rcp_M)
rcp_zonal_trend_S  = zonal_trend(rcp_S)

In [ ]:
f = plt.figure(figsize=(8,5))
plt.tick_params(labelsize=14)
plt.axhline(0, c='k', lw=.5)
plt.plot(ctrl_zonal_trend_G.TLAT_bins, ctrl_zonal_trend_G, c='k' , lw=3  , ls='--', label='CTRL global')
plt.plot(ctrl_zonal_trend_A.TLAT_bins, ctrl_zonal_trend_A, c='C0', lw=1.5, ls='--', label='CTRL Atlantic')
plt.plot(ctrl_zonal_trend_P.TLAT_bins, ctrl_zonal_trend_P, c='C1', lw=1.5, ls='--', label='CTRL Pacific')
plt.plot(ctrl_zonal_trend_I.TLAT_bins, ctrl_zonal_trend_I, c='C2', lw=1.5, ls='--', label='CTRL Indian')
plt.plot(ctrl_zonal_trend_S.TLAT_bins, ctrl_zonal_trend_S, c='C3', lw=1.5, ls='--', label='CTRL Southern')
plt.plot(ctrl_zonal_trend_M.TLAT_bins, ctrl_zonal_trend_M, c='C4', lw=1.5, ls='--', label='CTRL Mediterr.')

plt.plot(rcp_zonal_trend_G.TLAT_bins , rcp_zonal_trend_G , c='k' , lw=3  , ls='-' , label='RCP global')
plt.plot(rcp_zonal_trend_A.TLAT_bins , rcp_zonal_trend_A , c='C0', lw=1.5, ls='-' , label='RCP Atlantic')
plt.plot(rcp_zonal_trend_P.TLAT_bins , rcp_zonal_trend_P , c='C1', lw=1.5, ls='-' , label='RCP Pacific')
plt.plot(rcp_zonal_trend_I.TLAT_bins , rcp_zonal_trend_I , c='C2', lw=1.5, ls='-' , label='RCP Indian')
plt.plot(rcp_zonal_trend_S.TLAT_bins , rcp_zonal_trend_S , c='C3', lw=1.5, ls='-' , label='RCP Southern')
plt.plot(rcp_zonal_trend_M.TLAT_bins , rcp_zonal_trend_M , c='C4', lw=1.5, ls='-' , label='RCP Mediterr.')
plt.xticks(np.arange(-90,91,30))
plt.xlim((-92,92))
plt.xlabel('latitude', fontsize=16)
plt.ylabel('zonal OHC linear trend [J/m/yr]', fontsize=16)
plt.legend(fontsize=12, ncol=2, frameon=False)
plt.tight_layout()
plt.savefig(f'{path_results}/OHC/OHC_zonal_trend')

## latitude - depth

In [ ]:
ctrl_zonal_levels_trend_G = zonal_levels_trend(ctrl)
ctrl_zonal_levels_trend_A = zonal_levels_trend(ctrl_A)
ctrl_zonal_levels_trend_P = zonal_levels_trend(ctrl_P)
ctrl_zonal_levels_trend_I = zonal_levels_trend(ctrl_I)
ctrl_zonal_levels_trend_M = zonal_levels_trend(ctrl_M)
ctrl_zonal_levels_trend_S = zonal_levels_trend(ctrl_S)

rcp_zonal_levels_trend_G  = zonal_levels_trend(rcp)
rcp_zonal_levels_trend_A  = zonal_levels_trend(rcp_A)
rcp_zonal_levels_trend_P  = zonal_levels_trend(rcp_P)
rcp_zonal_levels_trend_I  = zonal_levels_trend(rcp_I)
rcp_zonal_levels_trend_M  = zonal_levels_trend(rcp_M)
rcp_zonal_levels_trend_S  = zonal_levels_trend(rcp_S)

In [ ]:
rcp_zonal_levels_trend_I.plot()

In [ ]:
f = plt.figure(figsize=(8,5))
plt.tick_params(labelsize=14)
X_ctrl, Y_ctrl = np.meshgrid(ctrl.TLAT_bins[11:], -tdepth/1e3)
maxv = 7e11
plt.pcolormesh(X_ctrl, Y_ctrl, ctrl_zonal_levels_trend_G.T, cmap=cmocean.cm.balance, vmin=-maxv, vmax=maxv)
plt.xticks(np.arange(-90,91,30))
plt.xlim((-92,92))
plt.xlabel('latitude', fontsize=16)
plt.ylabel('depth [km]', fontsize=16)
plt.colorbar()
plt.tight_layout()
plt.savefig(f'{path_results}/OHC/OHC_zonal_levels_trend_ctrl')

In [ ]:
f = plt.figure(figsize=(8,5))
plt.tick_params(labelsize=14)
X_rcp, Y_rcp = np.meshgrid(rcp.TLAT_bins[11:], -tdepth/1e3)
maxv = 7e11
plt.pcolormesh(X_rcp, Y_rcp, rcp_zonal_levels_trend_G.T, cmap=cmocean.cm.balance, vmin=-maxv, vmax=maxv)
plt.xticks(np.arange(-90,91,30))
plt.xlim((-92,92))
plt.xlabel('latitude', fontsize=16)
plt.ylabel('depth [km]', fontsize=16)
plt.colorbar(extend='max')
plt.tight_layout()
plt.savefig(f'{path_results}/OHC/OHC_zonal_levels_trend_rcp')

In [ ]:
f = plt.figure(figsize=(8,5))
plt.tick_params(labelsize=14)
X_rcp, Y_rcp = np.meshgrid(rcp.TLAT_bins[11:], -tdepth/1e3)
maxv = 10e11
plt.pcolormesh(X_rcp, Y_rcp, (rcp_zonal_levels_trend_G-ctrl_zonal_levels_trend_G).T, cmap=cmocean.cm.balance, vmin=-maxv, vmax=maxv)
plt.xticks(np.arange(-90,91,30))
plt.xlim((-92,92))
plt.xlabel('latitude', fontsize=16)
plt.ylabel('depth [km]', fontsize=16)
plt.colorbar(extend='max')
plt.tight_layout()
plt.savefig(f'{path_results}/OHC/OHC_zonal_levels_trend_rcp-ctrl')

In [ ]:
rcp_P.OHC_zonal_levels.mean(dim='time').plot()

In [ ]:
P_lat_0_100   = rcp_P.OHC_zonal_levels.sel({'z_t':slice(0,1e5)}).sum(dim='z_t')
P_lat_100_300 = rcp_P.OHC_zonal_levels.sel({'z_t':slice(1e5,3e5)}).sum(dim='z_t')

In [ ]:
rcp_times

In [ ]:
plt.plot(np.polyfit(rcp_times, P_lat_0_100, 1)[0])
plt.plot(np.polyfit(rcp_times, P_lat_100_300, 1)[0])

In [ ]:
P_lat_0_100_mean_trend   = np.polyfit(rcp_times, P_lat_0_100, 1)[0]
P_lat_100_300_mean_trend = np.polyfit(rcp_times, P_lat_100_300, 1)[0]
P_lat_0_100_mean_trend   = np.mean(P_lat_0_100_mean_trend[np.nonzero(P_lat_0_100_mean_trend)])
P_lat_100_300_mean_trend = np.mean(P_lat_100_300_mean_trend[np.nonzero(P_lat_100_300_mean_trend)])
print(f'{P_lat_0_100_mean_trend:.2e}, {P_lat_100_300_mean_trend:.2e}')

In [ ]:
P_lat_0_100_detr   = (P_lat_0_100   - P_lat_0_100.sel(  {'time':slice(730000,735000)}).mean(dim='time')).where(P_lat_0_100!=0)
P_lat_100_300_detr = (P_lat_100_300 - P_lat_100_300.sel({'time':slice(730000,735000)}).mean(dim='time')).where(P_lat_100_300!=0)
for t in range(len(rcp_times)):
    P_lat_0_100_detr[t,:]   =  P_lat_0_100_detr[t,:]   - P_lat_0_100_mean_trend*t
    P_lat_100_300_detr[t,:] =  P_lat_100_300_detr[t,:] - P_lat_100_300_mean_trend*t

In [ ]:
X, Y = np.meshgrid(rcp_times, P_lat_100_300_detr.TLAT_bins)

f, ax = plt.subplots(2,1,figsize=(8,8), sharex=True)

ax[0].pcolormesh(X, Y, P_lat_0_100_detr.T  , cmap=cmocean.cm.balance)
ax[1].pcolormesh(X, Y, P_lat_100_300_detr.T, cmap=cmocean.cm.balance)

ax[1].set_xlabel('time [years]', fontsize=16)
ax[0].text(.02, .9, '0-100 m'  , transform=ax[0].transAxes, fontsize=16)
ax[1].text(.02, .9, '100-300 m', transform=ax[1].transAxes, fontsize=16)
for i in range(2):
    ax[i].tick_params(labelsize=14)
    ax[i].set_ylim((-40,70))
    ax[i].set_ylabel('latitude', fontsize=16)
plt.tight_layout()
plt.savefig(f'{path_results}/OHC/OHC_Hovmoeller_lat_Pacific_rcp')

In [ ]:
P_lat_0_100.sel({'TLAT_bins':-5}, method='nearest').plot()


In [ ]:
P_lat_100_300.sel({'TLAT_bins':-5}, method='nearest').plot()

In [ ]:
P_lat_100_300.z_t

In [ ]:
# hiatus
X_rcp, Y_rcp = np.meshgrid(rcp.TLAT_bins, -tdepth[:20]/1e3)
maxv=8e13
plt.pcolormesh(X_rcp, Y_rcp, (rcp.OHC_zonal_levels[28,:20,:]-rcp.OHC_zonal_levels[20,:20,:]), cmap='RdBu_r', vmin=-maxv, vmax=maxv)
plt.colorbar()
plt.tight_layout()